In [ ]:
# Fix protobuf and TensorFlow compatibility issues
!pip uninstall protobuf -y --quiet
!pip install protobuf==3.20.3 --quiet
!pip install tensorflow --upgrade --quiet

print("Protobuf and TensorFlow updated. Please restart runtime if you encounter import errors.")

## Troubleshooting TensorFlow/Protobuf Errors

If you see `MessageFactory` errors or CUDA warnings:

1. **Restart the runtime**: Runtime → Restart Runtime
2. **Run the protobuf fix cell** (Cell 2) first
3. **Then run the install cell** (Cell 4)
4. **Restart again** if needed
5. **Continue with the rest of the cells**

The errors are due to version conflicts between TensorFlow and protobuf. The fixes above should resolve them.

In [ ]:
# Setup Kaggle API
from google.colab import files
print("Please upload your kaggle.json file:")
uploaded = files.upload()

# Setup Kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

print("Kaggle API setup complete!")


# # Setup Kaggle API (hardcoded credentials)
# import json
# import os

# # Create kaggle.json with credentials
# kaggle_credentials = {
#     "username": "xx",
#     "key": "xxx"
# }

# # Setup Kaggle directory
# os.makedirs(os.path.expanduser("~/.kaggle"), exist_ok=True)

# # Write credentials file
# kaggle_path = os.path.expanduser("~/.kaggle/kaggle.json")
# with open(kaggle_path, "w") as f:
#     json.dump(kaggle_credentials, f)

# # Set permissions
# os.chmod(kaggle_path, 0o600)

# print("Kaggle API setup complete!")

In [ ]:
# Install dependencies (run after protobuf fix)
!pip install rich scikit-learn tensorflow-addons --quiet

In [ ]:
# Check GPU availability and configure TensorFlow
import tensorflow as tf

print("=" * 60)
print("GPU DETECTION & CONFIGURATION")
print("=" * 60)

# List all GPUs
gpus = tf.config.list_physical_devices('GPU')
print(f"\nPhysical GPUs: {len(gpus)}")
for gpu in gpus:
    print(f"  - {gpu}")

# Check CUDA/cuDNN
if gpus:
    print("\n✅ GPU is AVAILABLE!")
    print(f"TensorFlow built with CUDA: {tf.test.is_built_with_cuda()}")
    print(f"GPU device available: {tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None)}")

    # Enable memory growth to prevent TensorFlow from allocating all GPU memory at once
    for gpu in gpus:
        try:
            tf.config.experimental.set_memory_growth(gpu, True)
            print(f"Memory growth enabled for {gpu.name}")
        except RuntimeError as e:
            print(f"Error setting memory growth: {e}")

    # Get GPU details
    print("\n📊 GPU Details:")
    gpu_details = tf.config.experimental.get_device_details(gpus[0])
    for key, value in gpu_details.items():
        print(f"  {key}: {value}")
else:
    print("\n❌ NO GPU DETECTED - Training will use CPU only!")
    print("In Colab: Go to Runtime → Change runtime type → Set Hardware accelerator to 'GPU' or 'T4 GPU'")

print("\n" + "=" * 60)
print(f"TensorFlow version: {tf.__version__}")
print("=" * 60)

In [ ]:
# Import libraries (with error handling)
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Suppress TensorFlow warnings

try:
    import argparse
    import json
    import shutil
    from datetime import datetime
    from typing import List, Dict
    from pathlib import Path

    import tensorflow as tf
    from tensorflow import keras

    # Suppress more warnings
    tf.get_logger().setLevel('ERROR')

    from rich import print
    from sklearn.model_selection import train_test_split
    from collections import Counter

    AUTOTUNE = tf.data.AUTOTUNE
    print("All imports successful!")
except Exception as e:
    print(f"Import error: {e}")
    # Fallback imports
    import argparse
    import json
    import shutil
    from datetime import datetime
    from typing import List, Dict
    from pathlib import Path

    import tensorflow as tf
    from tensorflow import keras
    tf.get_logger().setLevel('ERROR')

    from sklearn.model_selection import train_test_split
    from collections import Counter

    AUTOTUNE = tf.data.AUTOTUNE
    print("Basic imports successful (without rich)")

## ⚡ Performance Tips

**If training is slow:**

1. **Verify GPU is active**: Run the GPU check cell above - you should see "GPU is AVAILABLE"
2. **In Google Colab**: Runtime → Change runtime type → Hardware accelerator = **T4 GPU** (or A100/V100 if available)
3. **Batch size**: Increase from 64 to 128 or 256 if you have GPU memory available
4. **Mixed precision**: Enable by setting `mixed_precision=True` in config (2-3x speedup on modern GPUs)
5. **Check GPU utilization**: Run `!nvidia-smi` in a cell to see if GPU is being used during training

Expected speeds:
- **CPU only**: ~200-300 seconds/epoch (very slow)
- **T4 GPU**: ~30-50 seconds/epoch
- **V100/A100**: ~10-20 seconds/epoch

In [ ]:
# Configuration class
class TrainConfig:
    def __init__(self, **kwargs):
        self.image_size = kwargs.get('image_size', 224)
        self.batch_size = kwargs.get('batch_size', 64)
        self.epochs = kwargs.get('epochs', 15)
        self.validation_split = kwargs.get('validation_split', 0.2)
        self.fine_tune_from = kwargs.get('fine_tune_from', 100)
        self.mixed_precision = kwargs.get('mixed_precision', False)
        self.model_dir = kwargs.get('model_dir', '/content/models')
        self.datasets = kwargs.get('datasets', [])
        self.no_class_weights = kwargs.get('no_class_weights', False)
        self.repr_samples = kwargs.get('repr_samples', 100)
        self.brightness_factor = kwargs.get('brightness_factor', 0.1)
        self.contrast_factor = kwargs.get('contrast_factor', 0.1)
        self.seed = 42

In [ ]:
# Canonical classes mapping
def get_canonical_classes() -> list[str]:
    """Return the canonical waste classification classes."""
    return ["paper_cardboard", "glass", "recyclables", "bio_waste", "textile_reuse", "electronics", "battery", "residual_waste"]

In [ ]:
def consolidate_datasets(base_dir, merged_dir):
    """
    Consolidate multiple Kaggle datasets into a unified structure.
    Maps various class names to standardized categories.
    """
    import os
    import shutil
    from pathlib import Path

    # Define dataset configurations with their paths and class mappings
    datasets = [
        {
            'name': 'asdasdasasdas/garbage-classification',
            'slug': 'asdasdasasdas_garbage-classification',
            'path': 'Garbage classification/Garbage classification',
            'class_mappings': {
                'cardboard': 'paper_cardboard',
                'glass': 'glass',
                'metal': 'recyclables',
                'paper': 'paper_cardboard',
                'plastic': 'recyclables',
                'trash': 'residual_waste'
            }
        },
        {
            'name': 'glhdamar/new-trash-classfication-dataset',
            'slug': 'glhdamar_new-trash-classfication-dataset',
            'path': 'new-dataset-trash-type-v2',
            'class_mappings': {
                'cardboard': 'paper_cardboard',
                'glass': 'glass',
                'metal': 'recyclables',
                'paper': 'paper_cardboard',
                'plastic': 'recyclables',
                'trash': 'residual_waste',
                'e-waste': 'electronics',  # Map e-waste to electronics
                'organic': 'bio_waste',  # Map organic to bio_waste
                'textile': 'textile_reuse'   # Map textile to textile_reuse
            }
        },
        {
            'name': 'joebeachcapital/realwaste',
            'slug': 'joebeachcapital_realwaste',
            'path': 'realwaste-main/RealWaste',
            'class_mappings': {
                'Cardboard': 'paper_cardboard',
                'Glass': 'glass',
                'Metal': 'recyclables',
                'Paper': 'paper_cardboard',
                'Plastic': 'recyclables',
                'Miscellaneous Trash': 'residual_waste',
                'Food Organics': 'bio_waste',    # Map to bio_waste
                'Textile Trash': 'textile_reuse',    # Map to textile_reuse
                'Vegetation': 'bio_waste'        # Map to bio_waste
            }
        },
        {
            'name': 'mostafaabla/garbage-classification',
            'slug': 'mostafaabla_garbage-classification',
            'path': 'garbage_classification',
            'class_mappings': {
                'cardboard': 'paper_cardboard',
                'metal': 'recyclables',
                'paper': 'paper_cardboard',
                'plastic': 'recyclables',
                'trash': 'residual_waste',
                'battery': 'battery',      # Map battery to battery
                'biological': 'bio_waste',   # Map biological to bio_waste
                'clothes': 'textile_reuse',      # Map clothes to textile_reuse
                'shoes': 'textile_reuse',        # Map shoes to textile_reuse
                'brown-glass': 'glass',  # Map glass variants to glass
                'green-glass': 'glass',
                'white-glass': 'glass'
            }
        },
        {
            'name': 'karansolanki01/garbage-classification',
            'slug': 'karansolanki01_garbage-classification',
            'path': 'Data/Garbage_Classification',
            'class_mappings': {
                'Cardboard': 'paper_cardboard',
                'Glass': 'glass',
                'Metal': 'recyclables',
                'Paper': 'paper_cardboard',
                'Plastic': 'recyclables',
                'Battery': 'battery',   # Map Battery to battery
                'Clothes': 'textile_reuse'    # Map Clothes to textile_reuse
            }
        },
        {
            'name': 'sumn2u/garbage-classification-v2',
            'slug': 'sumn2u_garbage-classification-v2',
            'path': 'garbage-dataset',
            'class_mappings': {
                'cardboard': 'paper_cardboard',
                'glass': 'glass',
                'metal': 'recyclables',
                'paper': 'paper_cardboard',
                'plastic': 'recyclables',
                'trash': 'residual_waste',
                'battery': 'battery',     # Map battery to battery
                'biological': 'bio_waste',  # Map biological to bio_waste
                'clothes': 'textile_reuse',     # Map clothes to textile_reuse
                'shoes': 'textile_reuse'        # Map shoes to textile_reuse
            }
        }
    ]

    # Create consolidated directory structure
    consolidated_dir = Path(merged_dir)
    target_classes = ["paper_cardboard", "glass", "recyclables", "bio_waste", "textile_reuse", "electronics", "battery", "residual_waste"]

    for cls in target_classes:
        (consolidated_dir / cls).mkdir(parents=True, exist_ok=True)

    print(f"Consolidating datasets into {consolidated_dir}")
    total_images = 0

    for dataset in datasets:
        # Use slug for directory path (the actual extracted directory name)
        dataset_path = Path(base_dir) / dataset['slug'] / dataset['path']
        print(f"\nProcessing dataset: {dataset['name']}")
        print(f"Looking in: {dataset_path}")

        if not dataset_path.exists():
            print(f"Warning: Dataset path {dataset_path} does not exist, skipping...")
            continue

        # Process each source class
        for source_class, target_class in dataset['class_mappings'].items():
            source_dir = dataset_path / source_class
            target_dir = consolidated_dir / target_class

            if source_dir.exists():
                images = list(source_dir.glob('*'))
                print(f"  {source_class} -> {target_class}: {len(images)} images")

                for img_path in images:
                    if img_path.is_file() and img_path.suffix.lower() in ['.jpg', '.jpeg', '.png']:
                        shutil.copy2(img_path, target_dir / f"{dataset['slug']}_{img_path.name}")
                        total_images += 1
            else:
                print(f"  {source_class} -> {target_class}: directory not found")

    print(f"\nConsolidation complete! Total images copied: {total_images}")

    # Print final class counts
    print("\nFinal class distribution:")
    for cls in target_classes:
        count = len(list((consolidated_dir / cls).glob('*')))
        print(f"  {cls}: {count} images")

    return str(consolidated_dir)

In [ ]:
# Dataset building functions
def build_datasets(merged_dir: str, cfg: TrainConfig):
    image_size = cfg.image_size
    batch_size = cfg.batch_size
    seed = cfg.seed

    img_size = (image_size, image_size)
    class_names = get_canonical_classes()

    # Collect all files and labels
    all_files = []
    all_labels = []

    for class_idx, class_name in enumerate(class_names):
        class_dir = Path(merged_dir) / class_name
        if not class_dir.exists():
            continue
        for file in class_dir.glob("*"):
            if file.is_file() and file.suffix.lower() in ['.jpg', '.jpeg', '.png']:
                all_files.append(str(file))
                all_labels.append(class_idx)

    print(f"Found {len(all_files)} images across {len(class_names)} classes")

    # Stratified split
    label_counts = Counter(all_labels)
    min_samples_per_class = 10

    stratify_labels = [
        label if label_counts[label] >= min_samples_per_class else -1 for label in all_labels
    ]

    try:
        train_files, temp_files, train_labels, temp_labels = train_test_split(
            all_files, all_labels, test_size=0.3, stratify=stratify_labels, random_state=seed
        )
        val_files, test_files, val_labels, test_labels = train_test_split(
            temp_files,
            temp_labels,
            test_size=0.5,
            stratify=[
                label if label_counts[label] >= min_samples_per_class else -1
                for label in temp_labels
            ],
            random_state=seed,
        )
        print(f"Stratified split: {len(train_files)} train, {len(val_files)} val, {len(test_files)} test")
    except ValueError:
        train_files, temp_files, train_labels, temp_labels = train_test_split(
            all_files, all_labels, test_size=0.3, random_state=seed
        )
        val_files, test_files, val_labels, test_labels = train_test_split(
            temp_files, temp_labels, test_size=0.5, random_state=seed
        )
        print(f"Random split: {len(train_files)} train, {len(val_files)} val, {len(test_files)} test")

    # Augmentation
    brightness_factor = 0.1
    contrast_factor = 0.1

    aug = keras.Sequential([
        keras.layers.Resizing(image_size, image_size),
        keras.layers.RandomFlip("horizontal"),
        keras.layers.RandomRotation(0.1),
        keras.layers.RandomZoom(0.1),
        keras.layers.RandomContrast(contrast_factor),
        keras.layers.RandomBrightness(brightness_factor),
        keras.layers.RandomTranslation(0.05, 0.05),
    ])

    @tf.autograph.experimental.do_not_convert
    def load_train_image(file_path, label):
        img = tf.io.read_file(file_path)
        img = tf.image.decode_image(img, channels=3, expand_animations=False)
        img = aug(img)
        img = keras.applications.mobilenet_v2.preprocess_input(img)
        return img, tf.one_hot(label, len(class_names))

    @tf.autograph.experimental.do_not_convert
    def load_val_image(file_path, label):
        img = tf.io.read_file(file_path)
        img = tf.image.decode_image(img, channels=3, expand_animations=False)
        img = tf.image.resize(img, img_size)
        img = tf.cast(img, tf.float32)
        img = keras.applications.mobilenet_v2.preprocess_input(img)
        return img, tf.one_hot(label, len(class_names))

    train_ds = tf.data.Dataset.from_tensor_slices((train_files, train_labels))
    train_ds = train_ds.map(load_train_image, num_parallel_calls=AUTOTUNE)
    # Reduced shuffle buffer size to prevent memory issues
    train_ds = train_ds.shuffle(buffer_size=min(1000, len(train_files)), seed=seed)
    train_ds = train_ds.batch(batch_size).prefetch(AUTOTUNE)

    val_ds = tf.data.Dataset.from_tensor_slices((val_files, val_labels))
    val_ds = val_ds.map(load_val_image, num_parallel_calls=AUTOTUNE)
    val_ds = val_ds.batch(batch_size).prefetch(AUTOTUNE)

    test_ds = tf.data.Dataset.from_tensor_slices((test_files, test_labels))
    test_ds = test_ds.map(load_val_image, num_parallel_calls=AUTOTUNE)
    test_ds = test_ds.batch(batch_size).prefetch(AUTOTUNE)

    return train_ds, val_ds, test_ds, class_names, train_labels, val_labels, test_labels

In [ ]:
# Model building
def build_model(cfg: TrainConfig, class_names: list[str]):
    if cfg.mixed_precision:
        from tensorflow.keras import mixed_precision
        mixed_precision.set_global_policy("mixed_float16")

    base = keras.applications.MobileNetV2(
        input_shape=(cfg.image_size, cfg.image_size, 3), include_top=False, weights="imagenet"
    )

    if cfg.fine_tune_from is not None:
        base.trainable = True
        for layer in base.layers[: cfg.fine_tune_from]:
            layer.trainable = False
        lr = 1e-3
    else:
        base.trainable = False
        lr = 1e-3

    inputs = keras.Input(shape=(cfg.image_size, cfg.image_size, 3))
    x = base(inputs, training=True)
    x = keras.layers.GlobalAveragePooling2D()(x)
    x = keras.layers.Dropout(0.5)(x)
    x = keras.layers.Dense(256, activation="relu", kernel_regularizer=keras.regularizers.l2(0.01))(x)
    x = keras.layers.Dropout(0.3)(x)
    outputs = keras.layers.Dense(len(class_names), activation="softmax", dtype="float32")(x)

    model = keras.Model(inputs, outputs)

    try:
        opt = keras.optimizers.legacy.Adam(learning_rate=lr, clipnorm=1.0)
    except Exception:
        opt = keras.optimizers.Adam(learning_rate=lr, clipnorm=1.0)

    metrics = ["accuracy"]
    model.compile(optimizer=opt, loss="categorical_crossentropy", metrics=metrics)
    return model

In [ ]:
# Kaggle download function
def ensure_kaggle_download(datasets: list[str], raw_dir: str) -> str:
    """
    Download datasets from Kaggle if not already present.
    Returns the raw_dir path.
    """
    import os
    import zipfile
    import kaggle
    from pathlib import Path

    os.makedirs(raw_dir, exist_ok=True)

    for dataset in datasets:
        slug = dataset.replace("/", "_")
        zip_name = dataset.split("/")[-1] + ".zip"
        target_zip = Path(raw_dir) / zip_name
        target_dir = Path(raw_dir) / slug

        # If already extracted, skip everything
        if target_dir.exists():
            print(f"Dataset {dataset} already exists at {target_dir}, skipping download.")
            continue

        # If zip exists, skip download
        if not target_zip.exists():
            print(f"Downloading dataset: {dataset}")
            try:
                kaggle.api.dataset_download_files(dataset, path=raw_dir, quiet=False, unzip=False)
                print(f"Downloaded {dataset}")
            except Exception as e:
                print(f"Error downloading {dataset}: {e}")
                continue
        else:
            print(f"Zip already exists for {dataset}, skipping download.")

        # Extract to dataset-specific directory
        print(f"Extracting {target_zip} to {target_dir}...")
        try:
            with zipfile.ZipFile(target_zip, "r") as zf:
                zf.extractall(target_dir)
            print(f"Extracted {dataset}")
        except Exception as e:
            print(f"Error extracting {dataset}: {e}")

    return raw_dir

In [ ]:
# Main training function
def main():
    # Enable mixed precision for faster training on GPU (2-3x speedup)
    # This uses float16 for computations but float32 for variables to maintain accuracy
    if tf.config.list_physical_devices('GPU'):
        print("🚀 Enabling mixed precision training for GPU acceleration...")
        tf.keras.mixed_precision.set_global_policy('mixed_float16')
        print("✅ Mixed precision enabled: computations in float16, variables in float32")
    else:
        print("⚠️ Running on CPU - mixed precision disabled")

    # Configuration
    cfg = TrainConfig(
        image_size=224,
        batch_size=128,  # Increased from 64 - adjust based on GPU memory
        epochs=15,
        fine_tune_from=100,
        model_dir='/content/models',
        datasets=[
            "asdasdasasdas/garbage-classification",
            "glhdamar/new-trash-classfication-dataset",
            "joebeachcapital/realwaste",
            "mostafaabla/garbage-classification",
            "karansolanki01/garbage-classification",
            "sumn2u/garbage-classification-v2"
        ]
    )

    print(f"[bold cyan]Config:[/bold cyan]")
    for key, value in vars(cfg).items():
        print(f"  {key}: {value}")

    # Setup directories
    base_model_dir = cfg.model_dir
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    versioned_model_dir = os.path.join(base_model_dir, f"run_{timestamp}")
    cfg.model_dir = versioned_model_dir

    print(f"[bold green]Training Run:[/bold green] {timestamp}")

    # Prepare datasets (Colab downloads from Kaggle)
    raw_dir = "/content/raw_datasets"
    merged_dir = "/content/merged_dataset"

    print(f"[bold cyan]Downloading datasets from Kaggle[/bold cyan]")
    extracted = ensure_kaggle_download(cfg.datasets, raw_dir)
    merged = consolidate_datasets(extracted, merged_dir)

    # Build datasets
    train_ds, val_ds, test_ds, class_names, train_labels, val_labels, test_labels = build_datasets(merged, cfg)

    print(f"Dataset statistics:")
    print(f"  Classes: {class_names}")
    print(f"  Training samples: {len(train_labels)}")
    print(f"  Validation samples: {len(val_labels)}")
    print(f"  Test samples: {len(test_labels)}")

    # Class weights
    if cfg.no_class_weights:
        class_weight = None
    else:
        label_counts = Counter(train_labels)
        total = sum(label_counts.values())
        num_classes = len(class_names)
        class_weight = {}
        for i in range(num_classes):
            count = label_counts.get(i, 0)
            if count > 0:
                class_weight[i] = total / (num_classes * count)
            else:
                class_weight[i] = 10.0
        print(f"Class weights: {class_weight}")

    # Build model
    model = build_model(cfg, class_names)
    print(model.summary())

    # Callbacks
    callbacks = [
        keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True),
        keras.callbacks.ModelCheckpoint(
            os.path.join(cfg.model_dir, "waste_classifier_best.keras"), save_best_only=True
        ),
        keras.callbacks.ReduceLROnPlateau(
            monitor="val_accuracy", factor=0.5, patience=5, min_lr=1e-7, verbose=1
        ),
    ]
    os.makedirs(cfg.model_dir, exist_ok=True)

    # Save class weights
    weights_path = os.path.join(cfg.model_dir, "class_weights.json")
    with open(weights_path, "w") as f:
        if class_weight is not None:
            json.dump({class_names[i]: w for i, w in class_weight.items()}, f, indent=2)
        else:
            json.dump({"disabled": True}, f, indent=2)

    # Train
    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=cfg.epochs,
        callbacks=callbacks,
        class_weight=class_weight,
        verbose=1,  # Changed from 2 to 1 for progress bars
    )

    # Save model
    h5_path = os.path.join(cfg.model_dir, "waste_classifier.h5")
    model.save(h5_path)
    model.save(os.path.join(cfg.model_dir, "waste_classifier.keras"))
    print(f"Saved model to {h5_path}")

    # Evaluate
    print("Evaluating on test set...")
    test_loss, test_acc = model.evaluate(test_ds, verbose=1)
    print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")

    # Save labels
    labels_path = os.path.join(cfg.model_dir, "labels.json")
    with open(labels_path, "w") as f:
        json.dump(class_names, f)

    # Metadata
    metadata = {
        "timestamp": timestamp,
        "config": vars(cfg),
        "final_accuracy": float(test_acc),
        "final_loss": float(test_loss),
        "class_names": class_names,
        "training_samples": len(train_labels),
        "validation_samples": len(val_labels),
        "test_samples": len(test_labels),
    }

    metadata_path = os.path.join(versioned_model_dir, "training_metadata.json")
    with open(metadata_path, "w") as f:
        json.dump(metadata, f, indent=2, default=str)

    print(f"Training complete! Model saved in: {versioned_model_dir}")

In [ ]:
# Monitor GPU usage during training (run this in a separate cell while training)
!watch -n 1 nvidia-smi

## 🔍 Troubleshooting Slow Training

**If your training is as slow as CPU:**

### 1. Check GPU is Active
Run this command to verify GPU is being used:
```python
!nvidia-smi
```
You should see:
- GPU name (e.g., "Tesla T4" or "Tesla V100")
- GPU utilization % (should be 80-100% during training)
- Memory usage (should be several GB)

### 2. Verify Mixed Precision is Enabled
After running the main() function, you should see:
```
🚀 Enabling mixed precision training for GPU acceleration...
✅ Mixed precision enabled
```

### 3. Common Issues:

**❌ GPU shows 0% utilization during training:**
- The model might be running on CPU despite GPU being available
- Try restarting the runtime and running cells in order
- Check if TensorFlow sees the GPU: `tf.config.list_physical_devices('GPU')`

**❌ "ResourceExhausted" or "Out of Memory" errors:**
- Reduce batch size: Change `batch_size=128` to `batch_size=64` or `batch_size=32`
- The notebook will automatically retry with smaller batch

**❌ Training speed same as laptop (200-300 sec/epoch):**
- You're likely running on CPU
- **Colab users**: Go to **Runtime** → **Change runtime type** → Set **Hardware accelerator** to **GPU**
- Disconnect and reconnect to get a GPU instance

### 4. Expected Performance:
- **CPU only**: 200-300 seconds/epoch ⚠️ (Very slow)
- **T4 GPU**: 30-50 seconds/epoch ✅
- **V100/A100 GPU**: 10-20 seconds/epoch 🚀

In [ ]:
# Run the training
print("\n" + "="*80)
print("STARTING TRAINING")
print("="*80)

# Show GPU status before training
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"✅ Training will use GPU: {gpus[0].name}")
    print("💡 Tip: Run !nvidia-smi in another cell to monitor GPU usage in real-time")
else:
    print("⚠️ WARNING: No GPU detected! Training will be SLOW on CPU.")
    print("In Colab: Runtime → Change runtime type → GPU")

print("="*80 + "\n")

# Start training
main()

In [ ]:
# Download trained model (run after training completes)
from google.colab import files
import shutil

# Zip the models folder
shutil.make_archive('/content/models', 'zip', '/content/models')

# Download
files.download('/content/models.zip')

## Results

After training completes:
1. Run the download cell above to get your trained model
2. The model files will be in `run_YYYYMMDD_HHMMSS/` folder

Expected performance:
- First epoch accuracy: >0.3
- Final accuracy: 0.7-0.8
- Training time: ~30-60 minutes (Colab T4 GPU)

## Notes
- Colab sessions disconnect after 12 hours
- Save your work regularly
- Models are saved to `/content/models/`
- Use the download cell to get your trained model